In [8]:
'''
https://github.com/lschanne/melanoma-detection/blob/60ea8d7151fc754ca2f5522a0458c80abd065504/NN_on_image_features_and_metadata.ipynb
In [25]: preprocessor
Want to add this to model in In [102]

Solution:
https://machinelearningmastery.com/how-to-save-and-load-models-and-data-preparation-in-scikit-learn-for-later-use/
'''

from sklearn.pipeline import Pipeline
from sklearn.pipeline import  FeatureUnion
from sklearn.preprocessing import OneHotEncoder
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, LabelEncoder
from sklearn.model_selection import train_test_split
import pandas as pd
from pickle import dump

df = pd.read_csv(
    '/home/ubuntu/w210_melanoma/melanoma-detection/concatenated_whole.csv.gz', 
    compression='gzip',
    error_bad_lines=False,
)

drop_col = ['image_name','patient_id', 'diagnosis', 'benign_malignant']
data = df.drop(drop_col,axis = 1)
train, test = train_test_split(data, test_size = 0.2, random_state = 0)

X_train = train.drop(['target'], axis=1)
y_train = train['target']
X_test = test.drop(['target'], axis=1)
y_test = test['target']

cat = ['sex','anatom_site_general_challenge' ]
num = list(X_train.columns.values)
num = [elm for elm in num if elm not in cat]

class ColumnSelector(BaseEstimator, TransformerMixin):
    """Select only specified columns."""
    def __init__(self, columns):
        self.columns = columns

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[self.columns]

class Encoder(BaseEstimator, TransformerMixin):
    def __init__(self):
        self._encoders = {}
    
    def fit(self, X, y=None):
        for col in X.columns:
            encoder = LabelEncoder()
            encoder.fit(X[col])
            self._encoders[col] = encoder
        return self
    
    def transform(self, X):
        X_new = pd.DataFrame()
        for col, encoder in self._encoders.items():
            X_new[col] = encoder.transform(X[col])
        return X_new
        
cat_pipe = Pipeline([
    ('selector', ColumnSelector(cat)),
    ('label_encoder', Encoder()),
    ('imputer', KNNImputer(n_neighbors=1)),
    ('encoder', OneHotEncoder(handle_unknown='ignore', sparse=False))
])

# Define numerical pipeline
num_pipe = Pipeline([
    ('selector', ColumnSelector(num)),
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', MinMaxScaler())
])

# Combine categorical and numerical pipeline
preprocessor = FeatureUnion([
    ('cat', cat_pipe),
    ('num', num_pipe)
])

X_train_trasform = preprocessor.fit_transform(X_train)
# X_train_fin = pd.DataFrame(X_train_trasform)

X_test_trasform = preprocessor.transform(X_test)
X_test_fin = pd.DataFrame(X_test_trasform)

dump(preprocessor, open('/home/ubuntu/preprocessor.pkl', 'wb'))

In [12]:
from pickle import load
preprocessor2 = load(open('/home/ubuntu/preprocessor.pkl', 'rb'))
X_test_fin_2 = pd.DataFrame(preprocessor2.transform(X_test))
print((X_test_fin.values==X_test_fin_2.values).all())

True
